In [ ]:
#audio_chunks

In [ ]:
#np.shape(audio_chunks)

In [ ]:
#resample(audio_chunks, 16000/48000, 'sinc_best')

# Install

In [ ]:
#import os
#
#if not os.path.exists("INSTALLED"):
#    ! pip install -r requirements.txt > /dev/null && touch INSTALLED

In [ ]:
try:
    from ipdb import set_trace
except ImportError:
    ! pip install -r requirements.txt > /dev/null && touch INSTALLED

# Config

In [ ]:
DEFAULT_MODEL_SIZE = "base"
DEFAULT_DELAY = 2

# Main

In [ ]:
import os

from timeit import default_timer as timer
import time
from datetime import datetime
from datetime import timedelta

import traceback

from copy import copy

import numpy as np

import gradio as gr
import whisper
from whisper import tokenizer

from matplotlib import pyplot as plt

from samplerate.converters import resample

from scipy.io import wavfile

%load_ext gradio
%matplotlib widget 

In [ ]:
model = whisper.load_model(DEFAULT_MODEL_SIZE)

In [ ]:
audio_chunks = False
raw_chunks = False
audio_sr = False

In [ ]:
def reset_session():
    global audio_chunks
    global raw_chunks
    global audio_sr
    
    !rm /tmp/audio*.wav /tmp/chunks*.wav
    
    audio_chunks = np.array([], dtype='int16')
    raw_chunks = []
    audio_sr = False

In [ ]:
import IPython

def display_audio_chunk(index):
    global audio_chunks
    chunk = audio_chunks[index]['chunk']
    return IPython.display.Audio(chunk)

def display_audio_chunks():
    global audio_chunks
    for chunk in audio_chunks:
        IPython.display.display(IPython.display.Audio(chunk['chunk']))

In [ ]:
def get_full_transcript(model, audio_chunks):
    if len(audio_chunks) == 0:
        return ""
    
    chunk_texts = []
    
    for audio_chunk in audio_chunks:
        chunk_texts.append(audio_chunk['result'].text)
    
    return " ".join(chunk_texts)

In [ ]:
def write_audio_chunks_to_wav(chunks, sr, filename):
    scaled = np.int16(chunks/np.max(np.abs(chunks)) * 32767)
    wavfile.write(filename, sr, scaled)

In [ ]:
#!mediainfo /tmp/audio02qqtl0t.wav

In [ ]:
def transcribe_chunk(model, audio_chunks, audio):
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    #_, probs = model.detect_language(mel)
    #print(f"Detected language: {max(probs, key=probs.get)}")
    
    last_chunk = audio_chunks[:-1]
    
    if len(last_chunk) == 0:
        last_chunk = None
    else:
        last_chunk = last_chunk[0]
    
    # decode the audio
    options = whisper.DecodingOptions(
        language="en",
        prompt="" if not last_chunk else last_chunk['result'].text
        #suppress_tokens
    )
    
    return whisper.decode(model, mel, options)

In [ ]:
def get_timestamp():
    return datetime.utcnow().isoformat(sep=' ', timespec='milliseconds')

## Gradio

In [ ]:
#res = resample(audio_chunks, 16000/audio_sr, 'sinc_best')

In [ ]:
#a = whisper.pad_or_trim(audio_chunks.astype(np.float32))

In [ ]:
#mel = whisper.log_mel_spectrogram(a).to(model.device)

In [ ]:
#options = whisper.DecodingOptions(language="en")
#d = whisper.decode(model, mel, options)

In [ ]:
def get_full_transcript():
    global audio_chunks
    global audio_sr
    
    chunk_path = f"/tmp/chunks_{len(audio_chunks)}.wav"
    
    write_audio_chunks_to_wav(audio_chunks, audio_sr, chunk_path)
    
    audio = whisper.load_audio(chunk_path)
    audio = whisper.pad_or_trim(audio)
    
    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    #_, probs = model.detect_language(mel)
    #print(f"Detected language: {max(probs, key=probs.get)}")
    
    # decode the audio
    options = whisper.DecodingOptions(
        language="en"
        #prompt="" if not last_chunk else last_chunk['result'].text
        #suppress_tokens
    )
    
    return whisper.decode(model, mel, options)
    
    #local_chunks = audio_chunks.copy()
    #chunks_transformed = local_chunks.astype(np.float32)
    #chunks_transformed = local_chunks.astype(np.float32) / 32768.0
    
    #chunks_transformed = resample(local_chunks, 16000/audio_sr, 'sinc_best')
    
    result = model.transcribe(chunks_transformed, language="English")

    return result

In [ ]:
def get_full_transcript():
    global audio_chunks
    global audio_sr
    
    chunk_path = f"/tmp/chunks_{len(audio_chunks)}.wav"
    
    write_audio_chunks_to_wav(audio_chunks, audio_sr, chunk_path)
    
    result = model.transcribe(chunk_path, language="English")

    return result

In [ ]:
def transcribe(chunk, state, delay):
    #print(get_timestamp())
    
    #state['debug'] += f"{get_timestamp()}:\n"
    state['debug'] += f"."
    
    #print(chunk)
    #print(np.shape(chunk[1]))
    
    global audio_sr
    global audio_chunks
    global raw_chunks
    
    audio_sr = chunk[0]
    
    try:
        audio_chunks = np.append(audio_chunks, np.array(chunk[1], dtype='int16'))
        raw_chunks.append(chunk[1])
        print(chunk[1].dtype)
    except Exception:
        error_message = traceback.format_exc()
        print(error_message)
    
    transcript = get_full_transcript()
    
    #print(transcript)
    #print()
    #print(transcript["text"])
    #print()

    state['transcription'] = transcript['text']
    
    return state['transcription'], state['debug'], state

# Main

In [ ]:
reset_session()

print(audio_sr)
audio_chunks

In [ ]:
title = "two-wai"
description = "desc"

delay_slider = gr.Slider(minimum=0, maximum=10, value=DEFAULT_DELAY, label="Rate of transcription")

transcription_tb = gr.Textbox(label="Transcription", lines=5, max_lines=500)

debug_tb = gr.Textbox(label="Debug", lines=5, max_lines=200)

# FIX:
state = gr.State({"transcription": "", "debug": "", "audio_chunks": np.array([]), "audio_sr": 48000})

interface = gr.Interface(
    fn=transcribe,
    inputs=[
        # The format the audio file is converted to before being passed into the prediction function. "numpy" converts the audio to a tuple consisting of: (int sample rate, numpy.array for the data),
        # "filepath" passes a str path to a temporary file containing the audio.
        gr.Audio(source="microphone", type="numpy", streaming=True),
        state,
        delay_slider,
    ],
    outputs=[
        transcription_tb,
        debug_tb,
        state
    ],
    live=True,
    allow_flagging='auto',
    title=title,
    description=description,
).launch(
    enable_queue=True,
    debug=True,
    share=True,
    inline=True
)

# audio_chunks

In [ ]:
len(audio_chunks)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(np.array(raw_chunks).astype(np.float32), interpolation='nearest')
plt.show()

In [ ]:
np.avgerage(audio_chunks.astype(np.float32)  / 32768.0)

In [ ]:
model.transcribe(audio_chunks.astype(np.float32), language="English")

# Debug

In [ ]:
audio_chunks.dtype

In [ ]:
raw_chunks

In [ ]:
np.shape(audio_chunks)

In [ ]:
if len(audio_chunks) > 0:
    transcript = get_full_transcript(model, audio_chunks)
    print(transcript)
    display_audio_chunks()